# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path
import numpy as np

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities_f.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kapa'a,22.0752,-159.3190,25.99,82,24,0.45,US,1703907775
1,1,iqaluit,63.7506,-68.5145,-8.15,85,100,7.72,CA,1703907610
2,2,albany,42.6001,-73.9662,5.65,91,100,0.45,US,1703907776
3,3,ushuaia,-54.8000,-68.3000,7.81,87,40,1.54,AR,1703907556
4,4,nadi,-17.8000,177.4167,31.00,70,75,7.20,FJ,1703907776


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [19]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points("Lng", "Lat", geo = True, tiles = "OSM", frame_width = 700, frame_height = 500, size = "Humidity", scale = 0.9, color = "City", alpha = 0.5)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
data_filtered = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)]
data_filtered = data_filtered.loc[data_filtered["Wind Speed"] < 4.5]
data_filtered = data_filtered.loc[data_filtered["Cloudiness"] == 0]

# Drop any rows with null values
data_filtered = data_filtered.dropna(how='any')


# Display sample data
data_filtered

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
31,31,mancio lima,-7.6142,-72.8958,26.04,94,0,0.00,BR,1703907789
53,53,ciudad lazaro cardenas,17.9583,-102.2000,21.98,79,0,2.06,MX,1703907797
77,77,vila velha,-20.3297,-40.2925,26.08,86,0,3.60,BR,1703907808
154,154,coruripe,-10.1256,-36.1756,24.52,83,0,3.13,BR,1703907840
174,174,trujillo,-8.1160,-79.0300,21.98,83,0,4.12,PE,1703907848
180,180,san luis de la loma,17.2706,-100.8939,22.58,78,0,2.16,MX,1703907850
266,266,san pedro de macoris,18.4616,-69.2972,23.34,87,0,3.05,DO,1703907884
277,277,lazaro cardenas,17.9583,-102.2000,21.98,79,0,2.06,MX,1703907797
312,312,san sebastian de la gomera,28.0916,-17.1133,23.20,12,0,2.06,ES,1703907904
353,353,tura,25.5198,90.2201,23.00,50,0,0.40,IN,1703907927


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = data_filtered[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.insert(5, "Hotel Name", ' ')

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
31,mancio lima,BR,-7.6142,-72.8958,94,
53,ciudad lazaro cardenas,MX,17.9583,-102.2000,79,
77,vila velha,BR,-20.3297,-40.2925,86,
154,coruripe,BR,-10.1256,-36.1756,83,
174,trujillo,PE,-8.1160,-79.0300,83,
180,san luis de la loma,MX,17.2706,-100.8939,78,
266,san pedro de macoris,DO,18.4616,-69.2972,87,
277,lazaro cardenas,MX,17.9583,-102.2000,79,
312,san sebastian de la gomera,ES,28.0916,-17.1133,12,
353,tura,IN,25.5198,90.2201,50,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
limit = 20
params = {
    "categories": categories,
    "apiKey": geoapify_key,
    "limit": limit
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
   
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mancio lima - nearest hotel: No hotel found
ciudad lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
vila velha - nearest hotel: Hotel Prainha
coruripe - nearest hotel: No hotel found
trujillo - nearest hotel: Hostal Ayacucho
san luis de la loma - nearest hotel: No hotel found
san pedro de macoris - nearest hotel: No hotel found
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
san sebastian de la gomera - nearest hotel: Hotel Villa Gomera
tura - nearest hotel: No hotel found
bubaque - nearest hotel: Chez Julio
dwarka - nearest hotel: The Dwarika Hotel
freetown - nearest hotel: Formal Leona hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
31,mancio lima,BR,-7.6142,-72.8958,94,No hotel found
53,ciudad lazaro cardenas,MX,17.9583,-102.2000,79,Hotel Sol del Pacífico
77,vila velha,BR,-20.3297,-40.2925,86,Hotel Prainha
154,coruripe,BR,-10.1256,-36.1756,83,No hotel found
174,trujillo,PE,-8.1160,-79.0300,83,Hostal Ayacucho
180,san luis de la loma,MX,17.2706,-100.8939,78,No hotel found
266,san pedro de macoris,DO,18.4616,-69.2972,87,No hotel found
277,lazaro cardenas,MX,17.9583,-102.2000,79,Hotel Sol del Pacífico
312,san sebastian de la gomera,ES,28.0916,-17.1133,12,Hotel Villa Gomera
353,tura,IN,25.5198,90.2201,50,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [32]:
#Copy hotel df
filtered_hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity", "Hotel Name"]].copy()

#Remove no hotels found 
filtered_hotel_df = filtered_hotel_df.drop(filtered_hotel_df[filtered_hotel_df['Hotel Name'] == 'No hotel found'].index)
filtered_hotel_df



,City,Country,Lat,Lng,Humidity,Hotel Name
53,ciudad lazaro cardenas,MX,17.9583,-102.2000,79,Hotel Sol del Pacífico
77,vila velha,BR,-20.3297,-40.2925,86,Hotel Prainha
174,trujillo,PE,-8.1160,-79.0300,83,Hostal Ayacucho
277,lazaro cardenas,MX,17.9583,-102.2000,79,Hotel Sol del Pacífico
312,san sebastian de la gomera,ES,28.0916,-17.1133,12,Hotel Villa Gomera
394,bubaque,GW,11.2833,-15.8333,70,Chez Julio
547,dwarka,IN,22.2394,68.9678,51,The Dwarika Hotel
594,freetown,SL,8.4840,-13.2299,81,Formal Leona hotel


In [38]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = filtered_hotel_df.hvplot.points("Lng", "Lat", geo = True, tiles = "OSM", frame_width = 600, frame_height = 500, size = "Humidity", scale = 1.5, color = "City",
 alpha = 0.5, hover_cols = ["Hotel Name", "Country"])

# Display the map
hotel_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)